In [1]:
from AREgym import AREEnv

import gym


from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env

from stable_baselines3.ppo import MlpPolicy

from torch.utils.tensorboard import SummaryWriter
# env = env = gym.make('Pendulum-v0')
# env = AREEnv(250,10)
import numpy as np

# env = AREEnv(250, 10, num_laser_scan=10, heuristic_dist=10, save_map=True)
# # print(env.reset())
# # env.render()

# for i in range(2000):
#     env.step((np.random.rand() - 0.5) * 2)
# # env.laserscan
# print(env.step(1))
# env.render()



In [2]:
# env = AREEnv(250, 10, num_laser_scan=10, save_map=False, max_steps=128)
# env = AREEnv(500, 20, save_map=True)
# check_env(env)


In [6]:
from stable_baselines3.common.env_util import make_vec_env

env_kwargs = dict(grid=250, step_distance=10, num_laser_scan=10, save_map=False, max_steps=128)

env = make_vec_env(AREEnv, n_envs=4, env_kwargs=env_kwargs)


In [7]:
# policy_kwargs = dict(net_arch=dict(pi=[256, 128, 64], vf=[256, 128, 64]))
# import torch as th
policy_kwargs = dict(net_arch=[dict(pi=[128, 64], vf=[128, 64])])

In [8]:
model = PPO("MlpPolicy",
            env,
            verbose=0,
            n_steps=128,
            tensorboard_log='sb_runs/128',
            # batch_size=128,
            batch_size=512,
            policy_kwargs=policy_kwargs,
            seed=1
            )
# model = PPO("MlpPolicy", env, verbose=1, gamma=0.9)
model.learn(total_timesteps=1000000)
model.save("stable_baselines3_ppo")

/home/sheng/work/SARE-planner/AREgym.py:264: RuntimeWarning: invalid value encountered in multiply
  map_ = np.nan_to_num(((self.global_map - 1) * -np.inf).astype(np.float32), nan=1, posinf=np.inf)


In [9]:
model.save("stable_baselines3_ppo_128_1m")

In [ ]:
model = PPO.load("stable_baselines3_ppo")

In [7]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (shared_net): Sequential()
    (policy_net): Sequential(
      (0): Linear(in_features=20, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=20, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [14]:
env = AREEnv(250, 10, num_laser_scan=10, save_map=True, max_steps=128)

obs = env.reset()

for i in range(128):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    if dones:
        print('terminate')
        break
    print(i)
    # env.render()
env.render()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127


In [ ]:
print(env.reset())
env.render()

In [ ]:
print(obs)

In [ ]:
# import gym

# from stable_baselines3 import PPO
# from stable_baselines3.common.env_util import make_vec_env

# # Parallel environments
# env = make_vec_env("CartPole-v1", n_envs=4)

# model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=25000)
# model.save("ppo_cartpole")

# del model # remove to demonstrate saving and loading

# model = PPO.load("ppo_cartpole")

# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     env.render()